In [16]:
import importlib

import numpy as np

import pandas as pd

import rough_path_signatures_pricing.signature_regressor as sr
import rough_path_signatures_pricing.simulation_models as sm

importlib.reload(sm)
importlib.reload(sr)

<module 'rough_path_signatures_pricing.signature_regressor' from '/Users/arthur/Documents/Statistical Methods for Finance/rough-path-signatures-pricing-main 2/src/rough_path_signatures_pricing/signature_regressor.py'>

#### GBM Paths for Fitting Linear Functional with Signature to Approximate Payoffs

Create a generator object

In [ ]:
gbm_params = {
    "x0": 6944,
    "t0": 0,
    "t1": 1,
    "n_steps": 252,
    "seed": 42,
    "mu": 0,
    "sigma": 0.3,
}

In [18]:
gbm_simulator = sm.GBM(**gbm_params)

Creating the pricer object

In [19]:
N = 5
N_PATHS = 100


def asianPayoff(X: np.ndarray, K: float) -> np.ndarray:
    return np.maximum(np.mean(X, axis=1) - K, 0.0)

In [20]:
def barrier_up_out_call_payoff(path: np.ndarray, strike: float, barrier: float) -> float:
    """
    Compute the payoff of an up-and-out barrier call option.
    
    Parameters
    ----------
    path : np.ndarray
        A single simulated price path (1D array)
    strike : float
        Strike price of the call option
    barrier : float
        Barrier level (if price goes above this, option is knocked out)
    
    Returns
    -------
    float
        Payoff of the barrier call option
    """
    max_price = np.max(path)
    final_price = path[-1]
    
    if max_price > barrier:
        return 0.0
    
    # Otherwise, standard call option payoff at maturity
    return max(final_price - strike, 0.0)

def barrier_up_in_call_payoff(path: np.ndarray, strike: float, barrier: float) -> float:
    """
    Compute the payoff of an up-and-in barrier call option.
    
    Parameters
    ----------
    path : np.ndarray
        A single simulated price path (1D array)
    strike : float
        Strike price of the call option
    barrier : float
        Barrier level (price must hit this to activate the option)
    
    Returns
    -------
    float
        Payoff of the barrier call option
    """
    max_price = np.max(path)
    final_price = path[-1]
    
    # If the price NEVER reached the barrier, the option is worthless
    if max_price < barrier:
        return 0.0
    
    # If it DID hit the barrier, it behaves like a standard call option
    return max(final_price - strike, 0.0)

def asian_fixed_strike_call_payoff(
    path: np.ndarray, 
    strike: float
) -> float:
    """
    Compute the payoff of a fixed-strike Asian call option.
    
    Payoff = max(Average_Price - Strike, 0)
    """
    
    # Calculate the arithmetic average price
    average_price = np.mean(path, axis=1)
    
    # Standard call payoff logic using the average
    return np.maximum(average_price - strike, 0.0)

def asian_fixed_strike_put_payoff(
    path: np.ndarray, 
    strike: float
) -> float:
    """
    Compute the payoff of a fixed-strike Asian put option.
    
    Parameters
    ----------
    path : np.ndarray
        A single simulated price path.
    strike : float
        The pre-determined strike price.
    tenor_days : int
        The number of trading days in the averaging period.
    
    Returns
    -------
    float
        Payoff = max(Strike - Average_Price, 0)
    """
    
    # 2. Calculate the arithmetic average price over that period
    average_price = np.mean(path, axis=1)
    
    # 3. Put payoff logic: profit if average is below strike
    return np.maximum(strike - average_price, 0.0)

In [21]:
gbm_signature_pricer = sr.SignaturePricer(
    simulator=gbm_simulator, func=asian_fixed_strike_call_payoff, signature_degree=N
)

Fit the object to observed prices

In [22]:
df = pd.read_csv('synthetic_asian_prices.csv')

calls = df[df['Type'] == 'Asian_Call'].sort_values('Strike')
puts = df[df['Type'] == 'Asian_Put'].sort_values('Strike')

strikes = calls['Strike'].values
call_prices = calls['Price'].values
put_prices = puts['Price'].values

if np.array_equal(strikes, puts['Strike'].values):
    print("Strikes match perfectly for both calls and puts.")
    print(f"Number of data points: {len(strikes)}")
else:
    print("Warning: Strikes do not match between calls and puts.")

Strikes match perfectly for both calls and puts.
Number of data points: 50


In [25]:
gbm_signature_pricer.fit(call_prices, strikes, n_paths=N_PATHS)

Predict price on desired option with given parameter

In [27]:
STRIKE = np.array([7688])

In [28]:
gbm_signature_pricer.implied_expected_signature_.shape

(364,)

In [29]:
price = gbm_signature_pricer.predict(STRIKE)

In [30]:
price

np.float64(5788.952808705857)

## Below outdated, was made to simulate prices

#### Creating Basket of Similar Options

In [74]:
hullwhite_params = {"x0": 10, "t0": 0, "t1": 1, "n_steps": 100, "n_paths": 25}
hullwhite_seed = 42


def theta(t):
    return 0.03

In [75]:
hullwhite = sm.HullWhite(a=1, sigma=0.1, theta=theta, seed=hullwhite_seed)

In [76]:
hullwhite_paths, _, _ = hullwhite.simulate_paths(**hullwhite_params)
payoffs = hullwhite_paths

In [77]:
prices = asianPayoff(payoffs)
prices

array([6.33057037, 6.40023932, 6.37595713, 6.32474915, 6.31353483,
       6.37463307, 6.30025867, 6.31200737, 6.28925178, 6.26430336,
       6.29827663, 6.26328668, 6.29215938, 6.26486319, 6.35008527,
       6.27759942, 6.33549516, 6.34921802, 6.31743636, 6.26255873,
       6.26848975, 6.31656223, 6.28547231, 6.32479423, 6.25131481])

In [ ]:
gbm_signature_pricer.coeffs_.shape

(364,)

In [ ]:
gbm_signature_pricer.predict(prices)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 25 is different from 364)